In [1]:
import pandas as pd
import os
import glob
import numpy as np
from functools import reduce

In [2]:
os.chdir('/Users/rubencito/CAS_datascience/ADS_CAS_Bern_2020/Projects/M3/curated_datasets')
path = os.getcwd()
path

'/Users/rubencito/CAS_datascience/ADS_CAS_Bern_2020/Projects/M3/curated_datasets'

### Clean the revolution dataset

In [3]:
country_code = pd.read_csv(os.path.join('/Users/rubencito/CAS_datascience/ADS_CAS_Bern_2020/Projects/M3',
                                       "Country_code.csv"), encoding = "utf-8")
country_code.head()

,country_code,country_name
0,AFG,Afghanistan
1,ALB,Albania
2,DZA,Algeria
3,AND,Andorra
4,AGO,Angola


In [222]:
revo_raw = pd.read_csv(os.path.join(path, "Revolution Final.csv"), 
                       usecols = [ 2, 3, 4, 5], 
                       #dtype = {"year" : 'Int64'},
                       dtype = {"year" : 'int',
                               "type_of_change" : "int"},
                       encoding = "utf-8")
revo_col_names = revo_raw.columns.to_list()
revo_raw.head()

,type_of_change,year,Revolution,country_code
0,2,2017,True,TUR
1,7,2011,True,LBY
2,8,2011,True,EGY
3,10,2012,True,EGY
4,10,2012,True,EGY


In [206]:
revo_raw.type_of_change.unique()

array([ 2.,  7.,  8., 10.,  0.,  9.,  5., 11.])

In [207]:
country_code.shape

(195, 2)

In [208]:
len(country_code[country_code["country_code"].isin(revo_raw["country_code"])].country_code.unique())

32

In [209]:
len(revo_raw.country_code.unique())

32

In [210]:
revo_raw[~revo_raw["country_code"].isin(country_code["country_code"])]

,type_of_change,year,Revolution,country_code


In [211]:
rows_list = []

year_serie= pd.Series(range(2010,2021)).to_list()

my_codes = country_code.country_code.to_list()

for year in year_serie:
    
    for code in my_codes:

        my_dict = {}
        my_dict[year] = code
        my_dict.update(my_dict)
        
        rows_list.append(my_dict)
        


In [212]:
year_code_df = pd.DataFrame(rows_list).stack().to_frame().reset_index()
year_code_df.rename(columns = {"level_1" : "year",
                               0 : "country_code"}, inplace = True)
year_code_df = year_code_df[["year", "country_code"]]
year_code_df['year']=year_code_df['year'].astype("int")

year_code_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2145 entries, 0 to 2144
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   year          2145 non-null   int64 
 1   country_code  2145 non-null   object
dtypes: int64(1), object(1)
memory usage: 33.6+ KB


In [213]:
final_year_code_df = pd.merge(year_code_df, country_code,  how= "left", on = "country_code").reset_index(drop = True)
final_year_code_df.shape

(2145, 3)

In [125]:
final_year_code_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2145 entries, 0 to 2144
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   year          2145 non-null   int64 
 1   country_code  2145 non-null   object
 2   country_name  2145 non-null   object
dtypes: int64(1), object(2)
memory usage: 50.4+ KB


In [152]:
final_year_code_df[final_year_code_df["country_code"].isin(revo_raw["country_code"])]

,year,country_code,country_name
7,2010,ARM,Armenia
12,2010,BHR,Bahrain
20,2010,BOL,Bolivia (Plurinational State of)
26,2010,BFA,Burkina Faso
27,2010,BDI,Burundi
...,...,...,...
2128,2020,TUN,Tunisia
2129,2020,TUR,Turkey
2133,2020,UKR,Ukraine
2140,2020,VEN,Venezuela (Bolivarian Republic of)


In [149]:
final_year_code_df[(final_year_code_df["year"].isin(revo_raw["year"]))]

,year,country_code,country_name
0,2010,AFG,Afghanistan
1,2010,ALB,Albania
2,2010,DZA,Algeria
3,2010,AND,Andorra
4,2010,AGO,Angola
...,...,...,...
1750,2018,VEN,Venezuela (Bolivarian Republic of)
1751,2018,VNM,Viet Nam
1752,2018,YEM,Yemen
1753,2018,ZMB,Zambia


In [174]:
final_year_code_df[(final_year_code_df["year"].isin(revo_raw["year"])) & (final_year_code_df["country_code"].isin(revo_raw["country_code"]))].sort_values(by = ["country_code", "year"])

,year,country_code,country_name
7,2010,ARM,Armenia
202,2011,ARM,Armenia
397,2012,ARM,Armenia
592,2013,ARM,Armenia
787,2014,ARM,Armenia
...,...,...,...
972,2014,YEM,Yemen
1167,2015,YEM,Yemen
1362,2016,YEM,Yemen
1557,2017,YEM,Yemen


In [19]:
# save to csv the code_year dataset
final_year_code_df.to_csv('/Users/rubencito/CAS_datascience/ADS_CAS_Bern_2020/Projects/M3/final_year_code_df.csv', index = False)

In [25]:
final_year_code_df.isna().sum()

year            0
country_code    0
country_name    0
dtype: int64

In [ ]:
final_year_code_df

### NOTE: we must define criteria for filling NaNs in the Type of change dataset, and revolution datasets

In [59]:
revo_col_names[:2] + revo_col_names[3:]

['type_of_change', 'year', 'country_code']

In [51]:
revo_raw[revo_col_names[:]].head()

,type_of_change,year,Revolution,country_code
0,2,2017,True,TUR
1,7,2011,True,LBY
2,8,2011,True,EGY
3,10,2012,True,EGY
4,10,2012,True,EGY


In [172]:
revo_raw

,type_of_change,year,Revolution,country_code
0,2,2017,True,TUR
1,7,2011,True,LBY
2,8,2011,True,EGY
3,10,2012,True,EGY
5,0,2013,True,EGY
6,10,2014,True,EGY
7,9,2012,True,MMR
8,5,2012,True,YEM
9,5,2014,True,YEM
10,5,2015,True,YEM


## Must drop duplicates of revo data before the merge

In [214]:
revo_raw.shape

(56, 4)

In [215]:
revo_raw.drop_duplicates().shape

(53, 4)

In [216]:
revo_raw[revo_raw.duplicated()]

,type_of_change,year,Revolution,country_code
4,10.0,2012,True,EGY
11,5.0,2015,True,YEM
34,0.0,2012,True,MLI


In [225]:
revo_raw = revo_raw.drop_duplicates()
revo_raw.head()

,type_of_change,year,Revolution,country_code
0,2,2017,True,TUR
1,7,2011,True,LBY
2,8,2011,True,EGY
3,10,2012,True,EGY
5,0,2013,True,EGY


In [163]:
revo_raw.year.unique()

array([2017, 2011, 2012, 2013, 2014, 2015, 2018, 2010, 2016])

In [164]:
revo_raw.country_code.unique()

array(['TUR', 'LBY', 'EGY', 'MMR', 'YEM', 'TUN', 'BOL', 'CHN', 'HND',
       'HUN', 'MDG', 'RUS', 'VEN', 'THA', 'MYS', 'ARM', 'KGZ', 'UKR',
       'BDI', 'SOM', 'GMB', 'MLI', 'NER', 'CAF', 'BFA', 'GIN', 'CIV',
       'IRQ', 'FJI', 'SYC', 'SLB', 'BHR'], dtype=object)

In [165]:
revo_raw[revo_col_names[:2] + revo_col_names[3:]][revo_raw.duplicated()]

,type_of_change,year,country_code


In [166]:
revo_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 0 to 55
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   type_of_change  53 non-null     int64 
 1   year            53 non-null     int64 
 2   Revolution      53 non-null     bool  
 3   country_code    53 non-null     object
dtypes: bool(1), int64(2), object(1)
memory usage: 1.7+ KB


In [167]:
revo_raw.shape

(53, 4)

In [168]:
revo_col_names

['type_of_change', 'year', 'Revolution', 'country_code']

In [72]:
revo_col_names[:2] + revo_col_names[3:]

['type_of_change', 'year', 'country_code']

In [73]:
year_code_df.shape

(2145, 2)

In [74]:
revo_raw[revo_col_names[:2] + revo_col_names[3:]].duplicated().shape

(53,)

In [232]:
revo_raw[revo_col_names[:2] + revo_col_names[3:]].head()

,type_of_change,year,country_code
0,2,2017,TUR
1,7,2011,LBY
2,8,2011,EGY
3,10,2012,EGY
5,0,2013,EGY


In [226]:
#Extract df of type of change alone
#final_type_of_change_df = final_year_code_df.merge(revo_raw[revo_col_names[:2] + revo_col_names[3:]], how = "left").isna().sum()
#final_type_of_change_df.shape
final_type_of_change_df = final_year_code_df.merge(revo_raw[revo_col_names[:2] + revo_col_names[3:]], how = "left", on =  ["year", "country_code"]).sort_values(by = ["country_code", "year"])

In [227]:
final_type_of_change_df.shape

(2151, 3)

In [164]:
revo_raw.country_code.unique()

array(['TUR', 'LBY', 'EGY', 'MMR', 'YEM', 'TUN', 'BOL', 'CHN', 'HND',
       'HUN', 'MDG', 'RUS', 'VEN', 'THA', 'MYS', 'ARM', 'KGZ', 'UKR',
       'BDI', 'SOM', 'GMB', 'MLI', 'NER', 'CAF', 'BFA', 'GIN', 'CIV',
       'IRQ', 'FJI', 'SYC', 'SLB', 'BHR'], dtype=object)

In [230]:
final_type_of_change_df[final_type_of_change_df["country_code"] == "SLB"]

,year,country_code,type_of_change
162,2010,SLB,NaN
357,2011,SLB,NaN
553,2012,SLB,NaN
748,2013,SLB,NaN
944,2014,SLB,NaN
1141,2015,SLB,NaN
1336,2016,SLB,NaN
1531,2017,SLB,7.0
1726,2018,SLB,NaN
1921,2019,SLB,NaN


In [221]:
final_type_of_change_df.dropna(axis = 0).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 1573 to 1173
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            53 non-null     int64  
 1   country_code    53 non-null     object 
 2   type_of_change  53 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ KB


In [83]:
final_type_of_change_df.duplicated().any()

False

In [ ]:
final_type_of_change_df[final_type_of_change_df[]]

In [67]:
#Extract df of type of change alone
#final_type_of_change_df = final_year_code_df.merge(revo_raw[revo_col_names[:2] + revo_col_names[3:]], how = "left").isna().sum()
#final_type_of_change_df.shape
revo_raw[revo_col_names[:2] + revo_col_names[3:]].merge(final_year_code_df, how = "left", copy = False)

,type_of_change,year,country_code,country_name
0,2,2017,TUR,Turkey
1,7,2011,LBY,Libya
2,8,2011,EGY,Egypt
3,10,2012,EGY,Egypt
4,10,2012,EGY,Egypt
5,0,2013,EGY,Egypt
6,10,2014,EGY,Egypt
7,9,2012,MMR,Myanmar
8,5,2012,YEM,Yemen
9,5,2014,YEM,Yemen


In [26]:
revo_raw[revo_raw['country_code'].isin(final_year_code_df["country_code"])]

,type_of_change,year,Revolution,country_code
0,2,2017,True,TUR
1,7,2011,True,LBY
2,8,2011,True,EGY
3,10,2012,True,EGY
4,10,2012,True,EGY
5,0,2013,True,EGY
6,10,2014,True,EGY
7,9,2012,True,MMR
8,5,2012,True,YEM
9,5,2014,True,YEM


In [25]:
final_type_of_change_df[['type_of_change']] = final_type_of_change_df[['type_of_change']].fillna(value=False)# fill ,missing values with False
final_type_of_change_df[['type_of_change']] = final_type_of_change_df[['type_of_change']].astype(float)# convert values to float
cols_type = final_type_of_change_df.columns.to_list()
cols_type

['year', 'country_code', 'country_name', 'type_of_change']

In [20]:
cols_type[2:0:-1] + cols_type[0:4:3]

['country_name', 'country_code', 'year', 'type_of_change']

In [21]:
final_type_of_change_df = final_type_of_change_df[cols_type[2:0:-1] + cols_type[0:4:3]]
final_type_of_change_df

,country_name,country_code,year,type_of_change
0,Afghanistan,AFG,2010,0.0
1,Afghanistan,AFG,2011,0.0
2,Afghanistan,AFG,2014,0.0
3,Afghanistan,AFG,2015,0.0
4,Afghanistan,AFG,2017,0.0
...,...,...,...,...
2149,Zimbabwe,ZWE,2011,0.0
2150,Zimbabwe,ZWE,2019,0.0
2151,Zimbabwe,ZWE,2010,0.0
2152,Zimbabwe,ZWE,2016,0.0


In [22]:
#Extract df of revolution alone
final_revo_df = final_year_code_df.merge(revo_raw, how = "left")[["year", "country_code", "country_name", "Revolution"]]#.isna().sum()

In [23]:
final_revo_df[['Revolution']] = final_revo_df[['Revolution']].fillna(value=False)# fill ,missing values with False
final_revo_df[['Revolution']] = final_revo_df[['Revolution']].astype(float)# convert values to float
cols_revo = final_revo_df.columns.to_list()

In [24]:
cols_revo[2:0:-1] + cols_revo[0:4:3]

['country_name', 'country_code', 'year', 'Revolution']

In [25]:
final_revo_df = final_revo_df[cols_revo[2:0:-1] + cols_revo[0:4:3]]
final_revo_df

,country_name,country_code,year,Revolution
0,Afghanistan,AFG,2010,0.0
1,Afghanistan,AFG,2011,0.0
2,Afghanistan,AFG,2014,0.0
3,Afghanistan,AFG,2015,0.0
4,Afghanistan,AFG,2017,0.0
...,...,...,...,...
2149,Zimbabwe,ZWE,2011,0.0
2150,Zimbabwe,ZWE,2019,0.0
2151,Zimbabwe,ZWE,2010,0.0
2152,Zimbabwe,ZWE,2016,0.0


### Expor the final datasets

In [26]:
final_revo_df.to_csv(os.path.join(path, "Revolution_final_df.csv"), index = False)
final_type_of_change_df.to_csv(os.path.join(path, "Type_of_cchange_df.csv"), index = False)